In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os

# TO USE A DATABASE OTHER THAN SQLITE, USE THIS LINE
# Note that this is necessary for parallel execution amongst other things...
# os.environ['SNORKELDB'] = 'postgres:///snorkel-intro'

from snorkel import SnorkelSession
session = SnorkelSession()

In [ ]:
import numpy as np

# anns_folder = '/dfs/scratch0/paroma/coco/annotations/'
anns_folder = os.environ['SNORKELHOME'] + '/tutorials/babble/bike/data/'
train_path = anns_folder + 'train_anns.npy'
val_path = anns_folder + 'val_anns.npy'

train_anns = np.load(train_path).tolist()
val_anns = np.load(val_path).tolist()

In [ ]:
from snorkel.models import candidate_subclass

Biker = candidate_subclass('Biker', ['person', 'bike'])

In [ ]:
from snorkel.parser import ImageCorpusExtractor, CocoPreprocessor

corpus_extractor = ImageCorpusExtractor(candidate_class=Biker)

coco_preprocessor = CocoPreprocessor(train_path, source=0)
%time corpus_extractor.apply(coco_preprocessor)

coco_preprocessor = CocoPreprocessor(val_path, source=1)
%time corpus_extractor.apply(coco_preprocessor, clear=False)

for split in [0, 1]:
    num_candidates = session.query(Biker).filter(Biker.split == split).count()
    print("Split {} candidates: {}".format(split, num_candidates))

In [ ]:
labels_by_candidate = np.load(anns_folder + 'labels_by_candidate.npy').tolist()

In [ ]:
from snorkel.models import StableLabel
from snorkel.db_helpers import reload_annotator_labels

candidate_class = Biker
annotator_name = 'gold'

for candidate_hash, label in labels_by_candidate.items():
    set_name, image_idx, bbox1_idx, bbox2_idx = candidate_hash.split(':')
    source = {'train': 0, 'val': 1}[set_name]
    stable_id_1 = "{}:{}::bbox:{}".format(source, image_idx, bbox1_idx)
    stable_id_2 = "{}:{}::bbox:{}".format(source, image_idx, bbox2_idx)
    context_stable_ids = "~~".join([stable_id_1, stable_id_2])
    query = session.query(StableLabel).filter(StableLabel.context_stable_ids == context_stable_ids)
    query = query.filter(StableLabel.annotator_name == annotator_name)
    label = 1 if label else -1
    if query.count() == 0:
        session.add(StableLabel(
            context_stable_ids=context_stable_ids,
            annotator_name=annotator_name,
            value=label))

session.commit()
reload_annotator_labels(session, candidate_class, annotator_name, split=1, filter_label_split=False)

In [ ]:
stable_labels = session.query(StableLabel).filter(StableLabel.annotator_name == annotator_name).all()
len(stable_labels)

In [ ]:
candidates = num_candidates = session.query(Biker).filter(Biker.split == 1).all()
print(candidates[0])

In [ ]:
from snorkel.annotations import load_gold_labels

L_gold_dev = load_gold_labels(session, annotator_name='gold', split=1)
L_gold_dev

In [2]:
from snorkel.models import Bbox

In [8]:
A = Bbox(top=100, bottom=200, left=100, right=200)
B = Bbox(top=150, bottom=250, left=150, right=250)
C = Bbox(top=300, bottom=350, left=300, right=350)
D = Bbox(top=250, bottom=450, left=250, right=450)

In [9]:
A.area()

-10000

In [5]:
a.height()

9

In [6]:
a.width()

-9

In [7]:
a.area()

-81